In [2]:
import pandas as pd
import os

# QUESTION/ANSWER SCORE LIMIT
QUESTION_SCORE = 20
ANSWER_SCORE = 10


file = os.path.join('..','data', 'pandas-post-dataset.csv')


dataset = pd.read_csv(file, encoding='ISO-8859-1', error_bad_lines=False, sep=";")


In [3]:
from bs4 import BeautifulSoup
import io


# For every question
#index, row in df.iterrows()
# for idx, q in high_level_questions.iterrows():
#     print('Working on the question %s' % q.Id)
#     # Extract every answer
#     answers = all_posts[(all_posts.ParentId == q.Id) & (all_posts.Score > ANSWER_SCORE)]
    

#     # Filter the code in the answer by using the <code> tag
#     unformatted_codes = []
#     for idx, a in answers.iterrows():
#         soup = BeautifulSoup(a['Body'], "html.parser")
#         unformatted_codes.extend(soup.find_all('code'))
    
#     # Format from HTML to text
#     codes = [i.get_text() for i in unformatted_codes]
    
#     # Write it to a file
#     file = io.open(os.path.join('..', 'data', 'extracted-code',  str(q.Id)), 'a', encoding='ISO-8859-1')
#     for code in codes:
#         file.write("%s\n\n------------------------------------------------------------------\n\n" % code)
#     file.close()                  





In [4]:
from bs4 import BeautifulSoup
import io


# For every question
#index, row in df.iterrows()

def extract_code(df):

    soup = BeautifulSoup(df['Body'], "html.parser")
    unformatted_codes = soup.find_all('code')

    # Format from HTML to text
    codes = [i.get_text() for i in unformatted_codes]
    return codes
    

# Extract Code
dataset['Code'] = dataset.apply(extract_code, axis=1)

# Save to CSV
dataset_csv = dataset[['AnswerCount',
                                                 'CommentCount',
                                                 'Id',
                                                 'ParentId','PostTypeId','Score','Tags','Title',
                                                 'ViewCount','Code']]

dataset_csv.to_csv(os.path.join('..', 'data', 'pandas-code-dataset.csv'), encoding='ISO-8859-1', sep=";", doublequote=True)
# #high_level_questions_csv